In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from rich.console import Console
from rich.panel import Panel

console = Console()

# -------------------------------------------------
# CONFIG
# -------------------------------------------------
API_KEY = "YOUR_API_KEY_HERE"
BASE = "https://api.setlist.fm/rest/1.0"

HEADERS = {
    "x-api-key": API_KEY,
    "Accept": "application/json",
    "User-Agent": "ticket-hero-us-concert-harvester"
}

MAX_CALLS_PER_DAY = 1440       # YOUR FREE LIMIT
CALLS_MADE = 0
SLEEP_SECONDS = 0.6            # ~1.6 requests/second (safe)


# -------------------------------------------------
# RATE-LIMITED GET
# -------------------------------------------------
def safe_get(url):
    global CALLS_MADE

    if CALLS_MADE >= MAX_CALLS_PER_DAY:
        raise RuntimeError(f"Daily limit reached ({CALLS_MADE}/{MAX_CALLS_PER_DAY})")

    r = requests.get(url, headers=HEADERS)
    CALLS_MADE += 1

    # Progress heartbeat
    if CALLS_MADE % 50 == 0:
        console.print(f"[yellow]{CALLS_MADE} API calls used so far[/yellow]")

    # Respect API limits
    time.sleep(SLEEP_SECONDS)
    return r


# -------------------------------------------------
# HARVEST ALL US CONCERTS
# -------------------------------------------------
def fetch_us_concerts():
    console.print(
        Panel(
            "[cyan]Fetching ALL US concerts from Setlist.fm[/cyan]\n"
            "Rate-limited to 1440 requests/day",
            border_style="cyan",
        )
    )

    all_rows = []
    page = 1

    while True:
        url = f"{BASE}/search/setlists?countryCode=US&p={page}"
        r = safe_get(url)

        if r.status_code != 200:
            console.print(f"[red]Error {r.status_code}: {r.text}[/red]")
            break

        payload = r.json()
        setlists = payload.get("setlist", [])

        console.print(f"[green]Page {page}: {len(setlists)} concerts[/green]")

        # Stop when no more pages
        if not setlists:
            break

        # Extract structured concert metadata
        for s in setlists:
            venue = s.get("venue", {})
            city = venue.get("city", {})
            coords = city.get("coords", {})
            artist = s.get("artist", {})

            all_rows.append({
                "event_id": s.get("id"),
                "event_date": s.get("eventDate"),
                "event_last_updated": s.get("lastUpdated"),

                # Artist info
                "artist_name": artist.get("name"),
                "artist_mbid": artist.get("mbid"),

                # Tour
                "tour_name": s.get("tour", {}).get("name"),

                # Venue info
                "venue_name": venue.get("name"),
                "venue_id": venue.get("id"),
                "venue_city": city.get("name"),
                "venue_state": city.get("state"),
                "venue_country": city.get("country", {}).get("code"),
                "lat": coords.get("lat"),
                "lng": coords.get("long"),

                # Setlist & extras
                "sets": s.get("sets"),
                "info": s.get("info"),
                "url": s.get("url"),
            })

        page += 1

        # Absolute safety stop (should never hit)
        if page > 2000:
            console.print("[red]Stopping due to unusually large page count.[/red]")
            break

    df = pd.DataFrame(all_rows)
    return df


# -------------------------------------------------
# RUN HARVEST
# -------------------------------------------------
if __name__ == "__main__":
    df = fetch_us_concerts()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    path = f"setlistfm_us_concerts_{timestamp}.parquet"

    df.to_parquet(path, index=False)

    console.print(
        Panel(
            f"[bold green]US Concert Harvest Complete[/bold green]\n\n"
            f"Concerts collected: {len(df):,}\n"
            f"API calls used: {CALLS_MADE}/{MAX_CALLS_PER_DAY}\n"
            f"Saved → {path}",
            border_style="green",
        )
    )
